In [1]:
import os
import numpy as np
import pandas as pd
import pickle
#import torch
#from torch.utils.data import Dataset, DataLoader
from scipy.sparse import csr_matrix

In [2]:
df = pd.read_csv("usersha1-artmbid-artname-plays.tsv", sep='\t', header=None, names=['user-mboxsha1', 'artist_id', 'artist_name','plays'])

In [3]:
df

,user-mboxsha1,artist_id,artist_name,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
...,...,...,...,...
17535650,"sep 20, 2008",7ffd711a-b34d-4739-8aab-25e045c246da,turbostaat,12
17535651,"sep 20, 2008",9201190d-409f-426b-9339-9bd7492443e2,cuba missouri,11
17535652,"sep 20, 2008",e7cf7ff9-ed2f-4315-aca8-bcbd3b2bfa71,little man tate,11
17535653,"sep 20, 2008",f6f2326f-6b25-4170-b89d-e235b25508e8,sigur rós,10


In [4]:
df_info = pd.read_csv("usersha1-profile.tsv", sep='\t', header=None, names=['user-mboxsha1', 'gender', 'age','country','signup'])

In [5]:
df_info

,user-mboxsha1,gender,age,country,signup
0,00000c289a1829a808ac09c00daf10bc3c4e223b,f,22.0,Germany,"Feb 1, 2007"
1,00001411dc427966b17297bf4d69e7e193135d89,f,NaN,Canada,"Dec 4, 2007"
2,00004d2ac9316e22dc007ab2243d6fcb239e707d,NaN,NaN,Germany,"Sep 1, 2006"
3,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,m,19.0,Mexico,"Apr 28, 2008"
4,00007a47085b9aab8af55f52ec8846ac479ac4fe,m,28.0,United States,"Jan 27, 2006"
...,...,...,...,...,...
359342,fffe7823f67b433b45f22056467db921c1d3d7d0,m,25.0,Germany,"Jun 24, 2006"
359343,fffe8637bd8234309e871409c7ebef99a720afc1,m,25.0,Brazil,"Sep 9, 2007"
359344,fffe8c7f952d9b960a56ed4dcb40a415d924b224,m,20.0,United States,"Aug 8, 2007"
359345,ffff9af9ae04d263dae91cb838b1f3a6725f5ffb,m,20.0,Russian Federation,"Dec 3, 2005"


In [6]:
user_info = df_info[~df_info['gender'].isna()]
user_info

,user-mboxsha1,gender,age,country,signup
0,00000c289a1829a808ac09c00daf10bc3c4e223b,f,22.0,Germany,"Feb 1, 2007"
1,00001411dc427966b17297bf4d69e7e193135d89,f,NaN,Canada,"Dec 4, 2007"
3,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,m,19.0,Mexico,"Apr 28, 2008"
4,00007a47085b9aab8af55f52ec8846ac479ac4fe,m,28.0,United States,"Jan 27, 2006"
5,0000c176103e538d5c9828e695fed4f7ae42dd01,m,20.0,United Kingdom,"Jan 14, 2006"
...,...,...,...,...,...
359342,fffe7823f67b433b45f22056467db921c1d3d7d0,m,25.0,Germany,"Jun 24, 2006"
359343,fffe8637bd8234309e871409c7ebef99a720afc1,m,25.0,Brazil,"Sep 9, 2007"
359344,fffe8c7f952d9b960a56ed4dcb40a415d924b224,m,20.0,United States,"Aug 8, 2007"
359345,ffff9af9ae04d263dae91cb838b1f3a6725f5ffb,m,20.0,Russian Federation,"Dec 3, 2005"


In [7]:
user_info = user_info[~user_info['age'].isna()]
user_info

,user-mboxsha1,gender,age,country,signup
0,00000c289a1829a808ac09c00daf10bc3c4e223b,f,22.0,Germany,"Feb 1, 2007"
3,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,m,19.0,Mexico,"Apr 28, 2008"
4,00007a47085b9aab8af55f52ec8846ac479ac4fe,m,28.0,United States,"Jan 27, 2006"
5,0000c176103e538d5c9828e695fed4f7ae42dd01,m,20.0,United Kingdom,"Jan 14, 2006"
6,0000ee7dd906373efa37f4e1185bfe1e3f8695ae,m,17.0,Finland,"Nov 17, 2007"
...,...,...,...,...,...
359342,fffe7823f67b433b45f22056467db921c1d3d7d0,m,25.0,Germany,"Jun 24, 2006"
359343,fffe8637bd8234309e871409c7ebef99a720afc1,m,25.0,Brazil,"Sep 9, 2007"
359344,fffe8c7f952d9b960a56ed4dcb40a415d924b224,m,20.0,United States,"Aug 8, 2007"
359345,ffff9af9ae04d263dae91cb838b1f3a6725f5ffb,m,20.0,Russian Federation,"Dec 3, 2005"


In [8]:
user_info['gender'] = user_info['gender'].replace(['m', 'f'], [0, 1])

C:\Users\shaoyang\AppData\Local\Temp\ipykernel_24664\3192844797.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_info['gender'] = user_info['gender'].replace(['m', 'f'], [0, 1])


In [9]:
user_info['age'] = (user_info['age'] > 35).astype(int)

C:\Users\shaoyang\AppData\Local\Temp\ipykernel_24664\1217489010.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_info['age'] = (user_info['age'] > 35).astype(int)


In [10]:
user_info = user_info.drop(columns=['country','signup'])

In [11]:
num_rows = len(user_info)
user_info['user_id'] = list(range(1, num_rows + 1))
user_info

,user-mboxsha1,gender,age,user_id
0,00000c289a1829a808ac09c00daf10bc3c4e223b,1,0,1
3,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,0,0,2
4,00007a47085b9aab8af55f52ec8846ac479ac4fe,0,0,3
5,0000c176103e538d5c9828e695fed4f7ae42dd01,0,0,4
6,0000ee7dd906373efa37f4e1185bfe1e3f8695ae,0,0,5
...,...,...,...,...
359342,fffe7823f67b433b45f22056467db921c1d3d7d0,0,0,267379
359343,fffe8637bd8234309e871409c7ebef99a720afc1,0,0,267380
359344,fffe8c7f952d9b960a56ed4dcb40a415d924b224,0,0,267381
359345,ffff9af9ae04d263dae91cb838b1f3a6725f5ffb,0,0,267382


In [12]:
item_info = df.loc[:,['artist_id', 'artist_name']]
item_info = item_info.drop_duplicates(subset=['artist_id'])
item_info

,artist_id,artist_name
0,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch
1,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte
2,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge
3,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking
4,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks
...,...,...
17533401,fcf83c2c-dc9c-4f67-84f6-0fc5662fe816,ronny elliott
17533737,395473e8-90b3-4423-abbd-734c179921b2,magenta broken
17535001,32ccd3ed-7dad-4419-bc1b-8032bd3e1685,bound for attack
17535371,829a9c47-b3c6-4056-b5e6-962f9ab31157,sacred buddha


In [13]:
num_rows = len(item_info)
item_info['item_id'] = list(range(1, num_rows + 1))
item_info

,artist_id,artist_name,item_id
0,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,1
1,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,2
2,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,3
3,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,4
4,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,5
...,...,...,...
17533401,fcf83c2c-dc9c-4f67-84f6-0fc5662fe816,ronny elliott,160109
17533737,395473e8-90b3-4423-abbd-734c179921b2,magenta broken,160110
17535001,32ccd3ed-7dad-4419-bc1b-8032bd3e1685,bound for attack,160111
17535371,829a9c47-b3c6-4056-b5e6-962f9ab31157,sacred buddha,160112


In [14]:
df_ratings = df.loc[:,['user-mboxsha1', 'artist_id']]
df_ratings

,user-mboxsha1,artist_id
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8
...,...,...
17535650,"sep 20, 2008",7ffd711a-b34d-4739-8aab-25e045c246da
17535651,"sep 20, 2008",9201190d-409f-426b-9339-9bd7492443e2
17535652,"sep 20, 2008",e7cf7ff9-ed2f-4315-aca8-bcbd3b2bfa71
17535653,"sep 20, 2008",f6f2326f-6b25-4170-b89d-e235b25508e8


In [15]:
df_data = pd.merge(df_ratings,user_info,on="user-mboxsha1")
df_data = pd.merge(df_data,item_info,on="artist_id")
df_data

,user-mboxsha1,artist_id,gender,age,user_id,artist_name,item_id
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,1,0,1,betty blowtorch,1
1,0159d9ae1c8d76ab7fe3aa868a38e1da56cc06ab,3bd73256-3905-4f3a-97e2-8b341527f805,1,0,1422,betty blowtorch,1
2,0285490bb23814cf0dc18a5668261e243572d4eb,3bd73256-3905-4f3a-97e2-8b341527f805,1,0,2624,betty blowtorch,1
3,03d2e7186a6c29f281f6ff8b9bb2f7503822e7de,3bd73256-3905-4f3a-97e2-8b341527f805,1,0,3940,betty blowtorch,1
4,04e0999468b2bbd922f0fdd41ffd9a5f26f97609,3bd73256-3905-4f3a-97e2-8b341527f805,1,0,5000,betty blowtorch,1
...,...,...,...,...,...,...,...
13068497,fff820efe22db6c868515436de82af39e013b910,fcf83c2c-dc9c-4f67-84f6-0fc5662fe816,0,1,267342,ronny elliott,160109
13068498,fff9d5dfe2690a522bc4733f95a1a76cd41d8ead,395473e8-90b3-4423-abbd-734c179921b2,0,0,267348,magenta broken,160110
13068499,fffcdb342de39c80ca248877a3cd6ddccb2a1379,32ccd3ed-7dad-4419-bc1b-8032bd3e1685,0,0,267370,bound for attack,160111
13068500,fffe454af08a58c7bafe77491b586711f6b5e6f1,829a9c47-b3c6-4056-b5e6-962f9ab31157,0,0,267378,sacred buddha,160112


In [16]:
# Reduce item space by removing artists with few associated events
item_counts = df_data.item_id.value_counts()
filtered_items = item_counts.index[item_counts >= 110]
# Reduce item space by removing user with few associated events
user_counts = df_data.user_id.value_counts()
filtered_user = user_counts.index[user_counts >= 65]

In [17]:
df_data = df_data.loc[df_data.item_id.isin(filtered_items)]
df_data = df_data.loc[df_data.user_id.isin(filtered_user)]
df_data

,user-mboxsha1,artist_id,gender,age,user_id,artist_name,item_id
115,0140516a6b040b7658aef692ee3ee0abcfd4e539,f2fb0ff0-5679-42ec-a55c-15109ce6e320,0,0,1320,die Ärzte,2
208,04b74f5c5295922ecff07067f7eb179d81d655e2,f2fb0ff0-5679-42ec-a55c-15109ce6e320,1,0,4846,die Ärzte,2
212,04f09159115ea52259f0b9570208990bf8cdac1a,f2fb0ff0-5679-42ec-a55c-15109ce6e320,0,0,5066,die Ärzte,2
218,050c3931b9ae53772d075bf043c4fe3b54964ad2,f2fb0ff0-5679-42ec-a55c-15109ce6e320,1,0,5180,die Ärzte,2
292,078712db3aa957190c3409458f667a12c65a2060,f2fb0ff0-5679-42ec-a55c-15109ce6e320,0,0,7823,die Ärzte,2
...,...,...,...,...,...,...,...
12684866,969242d226c4717d28c6fdaa5ce5f0ced3e16e5a,c915ff9d-bde4-4250-be14-a0dbe4985eaa,0,0,156988,deastro,9665
12684875,9e88248d6804ef374eb438c69411ab432b9de27b,c915ff9d-bde4-4250-be14-a0dbe4985eaa,0,0,165201,deastro,9665
12684880,a6b8e107d412cda863a10b9d701e570efdd7a6e6,c915ff9d-bde4-4250-be14-a0dbe4985eaa,0,0,173582,deastro,9665
12684886,ac98fc9a318d1b15a25f45420b412b298bf7c686,c915ff9d-bde4-4250-be14-a0dbe4985eaa,1,0,179694,deastro,9665


In [18]:
ratings = df_data.loc[:,['user_id', 'item_id']]
user_info_0 = df_data.loc[:,['user_id', 'gender','age']].drop_duplicates(subset=['user_id'])
item_info_0 = df_data.loc[:,['item_id', 'artist_name']].drop_duplicates(subset=['item_id'])

In [24]:
ratings

,user_id,item_id
115,1320,2
208,4846,2
212,5066,2
218,5180,2
292,7823,2
...,...,...
12684866,156988,9665
12684875,165201,9665
12684880,173582,9665
12684886,179694,9665


In [22]:
user_info_0

,user_id,gender,age
115,1320,0,0
208,4846,1,0
212,5066,0,0
218,5180,1,0
292,7823,0,0
...,...,...,...
5037046,207004,0,1
5095666,195563,0,0
5639216,126932,0,0
6814771,751,0,0


In [25]:
user_info_0[user_info_0['gender']==0]

,user_id,gender,age
115,1320,0,0
212,5066,0,0
292,7823,0,0
328,9619,0,0
372,10911,0,0
...,...,...,...
5037046,207004,0,1
5095666,195563,0,0
5639216,126932,0,0
6814771,751,0,0


In [26]:
user_info_0[user_info_0['gender']==1]

,user_id,gender,age
208,4846,1,0
218,5180,1,0
456,15173,1,0
552,19627,1,0
872,32799,1,0
...,...,...,...
3048104,237434,1,1
3198310,176832,1,0
3532546,58093,1,1
3887724,6479,1,0


In [27]:
user_info_0[user_info_0['age']==0]

,user_id,gender,age
115,1320,0,0
208,4846,1,0
212,5066,0,0
218,5180,1,0
292,7823,0,0
...,...,...,...
3969908,201202,0,0
4174636,161504,0,0
5095666,195563,0,0
5639216,126932,0,0


In [28]:
user_info_0[user_info_0['age']==1]

,user_id,gender,age
913,34657,0,1
992,37516,0,1
1231,46940,1,1
1371,51884,0,1
1905,73577,1,1
...,...,...,...
3948106,139608,0,1
4171826,184914,0,1
4344501,103672,0,1
5037046,207004,0,1


In [23]:
item_info_0

,item_id,artist_name
115,2,die Ärzte
6826,3,melissa etheridge
7456,4,elvenking
8800,5,juliette & the licks
9856,6,red hot chili peppers
...,...,...
12619271,46607,priessnitz
12620334,14925,bligg
12648466,65520,ice nine kills
12682912,46672,freddie mcgregor


In [21]:
item_info_0.to_csv("item_info.csv", index=False)

In [222]:
user_info = user_info_0
item_info = item_info_0

In [223]:
def split_data(df, test_frac, on_user=True):
    if on_user == True:
        return split_on_user(df, test_frac)
    else:
        return split_on_item(df, test_frac)
def split_on_user(df, test_frac):
    # Identify properties and prepare split indices
    user_n_ratings = df.user_id.value_counts()
    n_users = user_n_ratings.shape[0]

    unique_users = user_n_ratings.index.values

    val_split_ind = int((1 - test_frac) * n_users)
    test_split_ind = int((1 - test_frac / 2) * n_users)

    user_idx = np.arange(n_users)
    np.random.shuffle(user_idx)

    # Identify users in each set
    train_users = unique_users[user_idx[:val_split_ind]]
    val_users = unique_users[user_idx[val_split_ind:test_split_ind]]
    test_users = unique_users[user_idx[test_split_ind:]]

    # Extract data from users
    train_df = df.loc[df["user_id"].isin(train_users), ["user_id", "item_id"]]
    val_df = df.loc[df["user_id"].isin(val_users), ["user_id", "item_id"]]
    test_df = df.loc[df["user_id"].isin(test_users), ["user_id", "item_id"]]

    return train_df, val_df, test_df


def split_on_item(df, test_frac, valtest=True):
    df_grouped_by_user = df.groupby("user_id")
    tr_list, val_list, te_list = [], [], []

    for i, (_, group) in enumerate(df_grouped_by_user):
        n_items_u = len(group)

        # Split the items of users that have rated more than 5 items
        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype="bool")
            n_valtest_items = int(test_frac * n_items_u)
            valtest_inds = np.random.choice(n_items_u, size=n_valtest_items, replace=False).astype("int64")
            idx[valtest_inds] = True

            tr_list.append(group[np.logical_not(idx)])
            if not valtest:
                te_list.append(group[idx])
                continue

            # Do a coin-toss in cases where the test items cannot be evenly split between val and test
            test_inds_start = n_valtest_items // 2 + (np.random.randint(0, 2) if n_valtest_items % 2 == 1 else 0)
            val_inds = valtest_inds[:test_inds_start]
            test_inds = valtest_inds[test_inds_start:]
            val_list.append(group.iloc[val_inds])
            te_list.append(group.iloc[test_inds])
        else:
            tr_list.append(group)

    df_tr = pd.concat(tr_list)
    df_val = pd.concat(val_list) if valtest else None
    df_te = pd.concat(te_list)

    return df_tr, df_val, df_te
def filter_and_reindex_items(train_df, val_df, test_df):
    # We can only process items seen in the training set
    unique_train_items = train_df["item_id"].unique()
    reindex_item = dict((iid, i) for (i, iid) in enumerate(unique_train_items))
    # reverse_map = dict((v, k) for k, v in reindex_item.items())

    def reindex(df, unique_train_items, item_map):
        # Filter
        df_copy = df.copy()
        df_copy = df_copy[df_copy["item_id"].isin(unique_train_items)]
        # Reindex
        df_copy["item_id"] = df_copy["item_id"].map(lambda x: item_map[x])
        return df_copy

    train_df = reindex(train_df, unique_train_items, reindex_item)
    val_df = reindex(val_df, unique_train_items, reindex_item)
    test_df = reindex(test_df, unique_train_items, reindex_item)
    return train_df, val_df, test_df  # , reverse_map


In [224]:
validation_frac = 0.2    
balance_sensitive = "True"
user_split = "True"
events = ratings

In [225]:
if user_split:
    if balance_sensitive:
        # Join in gender and age info temporarily
        events = events.join(user_info.set_index("user_id"), on="user_id")
        print(events.head(5))

        # Ensure similar compositions wrt sensitive attributes within each split
        train_df = val_df = test_df = pd.DataFrame()
        for gender_int in [0, 1]:
            for age_int in [0, 1]:
                tr_df, va_df, te_df = split_data(
                    events.query(f"gender == {gender_int} & age == {age_int}"),
                    validation_frac,
                    on_user=True,
                )
                train_df = pd.concat([train_df, tr_df])
                val_df = pd.concat([val_df, va_df])
                test_df = pd.concat([test_df, te_df])
    else:
        train_df, val_df, test_df = split_data(events, validation_frac, on_user=True)
else:
    train_df, val_df, test_df = split_data(events, validation_frac, on_user=False)

     user_id  item_id  gender  age
115     1320        2       0    0
208     4846        2       1    0
212     5066        2       0    0
218     5180        2       1    0
292     7823        2       0    0


In [226]:
# Save compressed processed data
unique_train_items = train_df["item_id"].unique()
with open("unique_items.txt", "w") as f:
    for item_id in unique_train_items:
        f.write(f"{item_id}\n")

In [227]:
train_df

,user_id,item_id
115,1320,2
212,5066,2
292,7823,2
328,9619,2
372,10911,2
...,...,...
12479291,265757,6633
12503058,44959,1549
12534125,102153,24603
12541402,245230,26782


In [228]:
train_df.to_csv("train.csv", index=False)

In [229]:
if user_split:
    # Split validation and test sets for vae evaluation
    val_tr, _, val_te = split_on_item(val_df, validation_frac, valtest=False)
    test_tr, _, test_te = split_on_item(test_df, validation_frac, valtest=False)

    val_tr.to_csv("val_tr.csv", index=False)
    val_te.to_csv("val_te.csv", index=False)
    test_tr.to_csv("test_tr.csv", index=False)
    test_te.to_csv("test_te.csv", index=False)
else:
    val_df.to_csv("val.csv", index=False)
    test_df.to_csv("test.csv", index=False)
    
user_info.to_csv("user_info.csv", index=False)